# CDAT Migration Regression Testing Notebook (`.nc` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How it works

It compares the relative differences (%) between ref and test variables between
the dev and `main` branches.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.
6. Review results for any outstanding differences (>=1e-5 relative tolerance).
   - Debug these differences (e.g., bug in metrics functions, incorrect variable references, etc.)


## Setup Code


In [44]:
import glob
from typing import List

import numpy as np
import xarray as xr

from e3sm_diags.derivations.derivations import DERIVED_VARIABLES

DEV_DIR = "843-migration-phase3-model-vs-model"
DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{DEV_DIR}/"
DEV_GLOB = sorted(glob.glob(DEV_PATH + "**/**/*.nc"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_DIR = "main-model-vs-model"
MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{MAIN_DIR}/"
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "**/**/*.nc"))
MAIN_NUM_FILES = len(MAIN_GLOB)


def _remove_unwanted_files(file_glob: List[str]) -> List[str]:
    """Remove files that we don't want to compare.

    * area_mean_time_series -- `main` does not generate netCDF
    * enso_diags -- `main` does not generate netCDF
    * qbo -- variable name differs
    * diurnal_cycle -- variable name differs
    * diff -- comparing the difference between regridded files is not helpful
      between branches because of the influence in floating point errors.

    Parameters
    ----------
    file_glob : List[str]
        _description_

    Returns
    -------
    List[str]
        _description_
    """

    new_glob = []

    for fp in file_glob:
        if (
            "area_mean_time_series" in fp
            or "enso_diags" in fp
            # or "qbo" in fp
            or "diurnal_cycle" in fp
            or "diff" in fp
        ):
            continue

        new_glob.append(fp)

    return new_glob


DEV_GLOB = _remove_unwanted_files(DEV_GLOB)
MAIN_GLOB = _remove_unwanted_files(MAIN_GLOB)

In [45]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_PATH and/or MAIN_PATH. "
            f"Please check {DEV_PATH} and {MAIN_PATH}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")


def _check_if_missing_files():
    missing_dev_files = []
    missing_main_files = []

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(MAIN_PATH, DEV_PATH)

        if fp_dev not in DEV_GLOB:
            missing_dev_files.append(fp_dev)

    for fp_dev in DEV_GLOB:
        fp_main = fp_dev.replace(DEV_PATH, MAIN_PATH)

        if fp_main not in MAIN_GLOB:
            missing_main_files.append(fp_main)

    return missing_dev_files, missing_main_files

In [46]:
def _get_relative_diffs():
    # We are mainly focusing on relative tolerance here (in percentage terms).
    atol = 0
    rtol = 1e-5

    results = {
        "missing_files": [],
        "matching_files": [],
        "mismatch_errors": [],
        "not_equal_errors": [],
        "key_errors": [],
    }

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(f"main/{MAIN_DIR}", DEV_DIR)

        print("Comparing:")
        print(f"    * {fp_dev}")
        print(f"    * {fp_main}")

        try:
            ds1 = xr.open_dataset(fp_dev)
            ds2 = xr.open_dataset(fp_main)
        except FileNotFoundError as e:
            print(f"    {e}")

            if isinstance(e, FileNotFoundError):
                results["missing_files"].append(fp_dev)

            continue

        if "qbo" in fp_main:
            dev_data = ds1["U"].values
            main_data = ds2["U"].values
        else:
            var_key = fp_main.split("-")[-3]
            # for 3d vars such as T-200
            var_key.isdigit()
            if var_key.isdigit():
                var_key = fp_main.split("-")[-4]

            dev_data = _get_var_data(ds1, var_key)
            main_data = _get_var_data(ds2, var_key)

        print(f"    * var_key: {var_key}")

        if dev_data is None or main_data is None:
            print("    * Could not find variable key in the dataset(s)")
            continue

        try:
            np.testing.assert_allclose(
                dev_data,
                main_data,
                atol=atol,
                rtol=rtol,
            )
            results["matching_files"].append(fp_main)
        except (KeyError, AssertionError) as e:
            msg = str(e)

            print(f"    {msg}")
            if "mismatch" in msg:
                results["mismatch_errors"].append(fp_main)
            elif "Not equal to tolerance" in msg:
                results["not_equal_errors"].append(fp_main)
        else:
            print(f"    * All close and within relative tolerance ({rtol})")

    return results


def _get_var_data(ds: xr.Dataset, var_key: str) -> np.ndarray:
    """Get the variable data using a list of matching keys.

    The `main` branch saves the dataset using the original variable name,
    while the dev branch saves the variable with the derived variable name.
    The dev branch is performing the expected behavior here.

    Parameters
    ----------
    ds : xr.Dataset
        _description_
    var_key : str
        _description_

    Returns
    -------
    np.ndarray
        _description_
    """

    data = None

    try:
        var_keys = DERIVED_VARIABLES[var_key].keys()
    except KeyError:
        var_keys = DERIVED_VARIABLES[var_key.upper()].keys()

    var_keys = [var_key] + list(sum(var_keys, ()))

    for key in var_keys:
        if key in ds.data_vars.keys():
            data = ds[key].values
            break

    return data

## 1. Check for matching and equal number of files


In [47]:
_check_if_files_found()

In [48]:
DEV_GLOB = [fp for fp in DEV_GLOB if "diff.nc" not in fp]
MAIN_GLOB = [fp for fp in MAIN_GLOB if "diff.nc" not in fp]

In [49]:
len(DEV_GLOB), len(MAIN_GLOB)

(144, 128)

In [50]:
missing_dev_files, missing_main_files = _check_if_missing_files()

print(f"Missing dev files: {len(missing_dev_files)}")
print(f"Missing main files: {len(missing_main_files)}")

Missing dev files: 0
Missing main files: 16


### Check missing dev files:


In [51]:
missing_dev_files

[]

### Check missing main files:

Results:

- `main` no files for variables:

  - `lat_lon`

```python
['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-200-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-200-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-500-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-500-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-850-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-850-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-T-200-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-T-200-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-T-850-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-T-850-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-U-200-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-U-200-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-U-850-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-U-850-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-Z3-500-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-Z3-500-JJA-global_test.nc']
```


In [52]:
missing_main_files

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-200-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-200-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-500-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-500-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-850-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-850-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-T-200-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-T-200-JJA-global_test.nc',
 '/global/cfs/cd

## 2 Compare the netCDF files between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [53]:
results = _get_relative_diffs()

Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-ALBEDO-ANN-global_test.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-ALBEDO-ANN-global_test.nc
    * var_key: ALBEDO
    * All close and within relative tolerance (1e-05)
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-ALBEDO-JJA-global_test.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-ALBEDO-JJA-global_test.nc
    * var_key: ALBEDO
    * All close and within relative tolerance (1e-05)
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-ALBEDOC-ANN-global_test.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-ALBEDOC-ANN-global_test.nc
    * var_key: ALBEDOC
    * All close and within relative tolerance (1e-05

In [54]:
(
    missing_files,
    matching_files,
    mismatch_errors,
    not_equal_errors,
    key_errors,
) = results.values()

In [55]:
sum_files_compared = (
    len(matching_files)
    + len(mismatch_errors)
    + len(not_equal_errors)
    + len(key_errors)
    + len(missing_files)
)

### Close results


In [56]:
pct_match = (len(matching_files) / sum_files_compared) * 100

print(
    f"Matching files count: {len(matching_files)}/{sum_files_compared}, {pct_match:.2f}%"
)

Matching files count: 128/128, 100.00%


In [57]:
pct_missing = (len(missing_files) / sum_files_compared) * 100

print(
    f"Missing files count: {len(missing_files)}/{sum_files_compared}, {pct_missing:.2f}%"
)

Missing files count: 0/128, 0.00%


### `NaN` Mismatching Errors (Not concerned)

I found these `nan` mismatch errors occur due to either:

1. Regional subsetting on "ccb" flag in CDAT adding a coordinate points -- removing these coordinates results in matching results
2. Slightly different masking in the data between xCDAT and CDAT via xESMF/ESMF -- same number of nans just slightly shifted over some coordinates points

- Refer to PR [#794](https://github.com/E3SM-Project/e3sm_diags/pull/794)


In [58]:
pct_mismatch = (len(mismatch_errors) / sum_files_compared) * 100

print(
    f"Mismatch errors count: {len(mismatch_errors)}/{sum_files_compared}, {pct_missing:.2f}%"
)

Mismatch errors count: 0/128, 0.00%


### Not Equal Errors (Not concerned)

Refer to GH PR [#794](https://github.com/E3SM-Project/e3sm_diags/pull/794)

> The following variables have large diffs due to a bug on main: MISRCOSP-CLDLOW_TAU1.3_9.4_MISR, MISRCOSP-CLDLOW_TAU1.3_MISR, MISRCOSP-CLDLOW_TAU9.4_MISR. I believe this branch is doing the correct thing. We can revisit the comparison for these variables once the issue below is fixed on main.
>
> [\[Bug\]: Silent bug in adjust_prs_val_units() conditional where if prs_val0: will be False if prs_val0 is 0 #797](https://github.com/E3SM-Project/e3sm_diags/issues/797)


In [59]:
pct_not_equal_errors = (len(not_equal_errors) / sum_files_compared) * 100

print(
    f"Not equal to tolerance count: {len(not_equal_errors)}/{sum_files_compared}, {pct_not_equal_errors:.2f}%"
)

Not equal to tolerance count: 0/128, 0.00%


In [60]:
not_equal_errors

[]

### Key Errors (None)


In [61]:
print(f"Key errors count: {len(key_errors)}")

Key errors count: 0


### Results

- 124/128 variables within rtol
- 4/128 not within rtol, but not of concern because there is a bug on `main`
  - TODO: Investigate the missing `lat_lon` variables on `main`

```python
['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-200-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-200-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-500-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-500-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-850-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-OMEGA-850-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-T-200-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-T-200-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-T-850-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-T-850-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-U-200-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-U-200-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-U-850-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-U-850-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-Z3-500-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-model-vs-model/lat_lon/model_vs_model/-Z3-500-JJA-global_test.nc']
```
